<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch03_SparkSQL_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
from pyspark_helpers import *
sc, spark, conf = initspark()


initializing pyspark
pyspark initialized


### Homework ##

### First Challenge
Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using SparkSql as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

### Second Challenge
Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


In [2]:
o = spark.read.csv(f'{datapath}/northwind/CSVHeaders/orders', header = True, inferSchema = True)
od = spark.read.csv(f'{datapath}/northwind/CSVHeaders/orderdetails', header = True, inferSchema = True)

o.createOrReplaceTempView('Orders')
od.createOrReplaceTempView('OrderDetails')
sql = """
select o.OrderID, o.CustomerID, o.OrderDate
           , COLLECT_SET(NAMED_STRUCT("ProductID", od.ProductID, 
                                      "UnitPrice", od.UnitPrice,
                                      "Quantity", od.Quantity,
                                      "Discount", od.discount)) as LineItems
from Orders as o join OrderDetails as od on o.OrderID = od.OrderID
GROUP BY o.OrderID, o.CustomerID, o.OrderDate
ORDER BY o.OrderID
"""
od2 = spark.sql(sql)
od2.write.mode('overwrite').json(f'{rootpath}Orders_LineItems.json')
print('Done')



Done


In [ ]:
shippers = spark.read.csv(f'{datapath}/northwind/TSV/shippers', header=False, sep = '\t')
#shippers.show()
print(shippers)
shippers.createOrReplaceTempView('shippers')
print(shippers.collect())
shippers = spark.sql('select cast(_c0 as int) as ShipperID, _c1 as ShipperName, _c2 as Phone from shippers')
shippers.createOrReplaceTempView('shippers')
print(shippers)

def isEven(x):
    return x % 2 == 0

from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark.udf.register('isEven', isEven, BooleanType())

spark.sql('select * FROM Shippers WHERE isEven(ShipperID)').show()


In [6]:
from pyspark.sql.functions import avg

o = spark.read.json(f'{rootpath}/Orders_LineItems.json')
display(o)
o.createOrReplaceTempView('Orders')
sql = '''
select OrderId, CustomerID, OrderDate, l.ProductID, l.UnitPrice, l.Quantity, l.Discount
FROM Orders LATERAL VIEW EXPLODE(LineItems) EXPLODED_TABLE AS l
'''
o2 = spark.sql(sql)
#display(o2)

o2.groupby('ProductID').agg(avg('UnitPrice')).show()

# o2.createOrReplaceTempView('FlatOrders')
# spark.sql('select avg(UnitPrice) from FlatOrders').show()


,CustomerID,LineItems,OrderDate,OrderID
0,SIMOB,"[(0.05, 16, 14, 17.45)]",1998-05-06T00:00:00.000Z,11074
1,RICSU,"[(0.15, 76, 2, 18.0), (0.15, 2, 10, 19.0), (0....",1998-05-06T00:00:00.000Z,11075
2,BONAP,"[(0.25, 19, 10, 9.2), (0.25, 14, 20, 23.25), (...",1998-05-06T00:00:00.000Z,11076
3,RATTC,"[(0.03, 16, 2, 17.45), (0.1, 8, 2, 40.0), (0.2...",1998-05-06T00:00:00.000Z,11077
4,FURIB,"[(0.15, 16, 40, 17.45), (0.15, 35, 20, 18.0), ...",1997-05-28T00:00:00.000Z,10551
5,HILAA,"[(0.0, 69, 18, 36.0), (0.0, 75, 30, 7.75)]",1997-05-29T00:00:00.000Z,10552
6,WARTH,"[(0.0, 31, 30, 12.5), (0.0, 16, 14, 17.45), (0...",1997-05-30T00:00:00.000Z,10553
7,OTTIK,"[(0.05, 23, 20, 9.0), (0.05, 77, 10, 13.0), (0...",1997-05-30T00:00:00.000Z,10554
8,SAVEA,"[(0.2, 56, 40, 38.0), (0.2, 24, 18, 4.5), (0.2...",1997-06-02T00:00:00.000Z,10555
9,ISLAT,"[(0.0, 2, 10, 19.0), (0.0, 13, 10, 6.0), (0.0,...",1998-01-13T00:00:00.000Z,10829


+---------+------------------+
|ProductID|    avg(UnitPrice)|
+---------+------------------+
|       26|28.856250000000003|
|       29|116.04312499999999|
|       65|          19.45625|
|       19| 8.532432432432433|
|       54| 6.804166666666667|
|       22|20.400000000000002|
|       77|12.110526315789473|
|        7| 29.17241379310345|
|       34|12.968421052631578|
|       50|             14.95|
|       57|18.143478260869564|
|       32|30.719999999999995|
|       43|43.042857142857144|
|       31|11.666666666666666|
|       39|             16.68|
|       25|13.066666666666666|
|       71|19.759523809523813|
|        6|24.166666666666668|
|       68|11.544117647058824|
|       72| 32.03684210526316|
+---------+------------------+
only showing top 20 rows

